In [5]:
from __future__ import division
import random
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from sklearn import preprocessing

In [6]:
def contActions2(x,demand,pv_gen,bat_t0,elprices,battery,const1,const2):
    charOrDisch = battery.contAct
    contChar=0
    contDisch=0
    lowBound=battery.capacity*0.2
    upBound=battery.capacity*0.8
    dim = len(x)
    bat_action = x[0]-bat_t0
    if bat_action>5:
        bat_action=5
        x[0]=bat_t0+5
    if bat_action<-5:
        bat_action=-5
        x[0]=bat_t0-5
        
    if charOrDisch == 1:
        if bat_action > 0:
            if x[0] == upBound:
                contChar+=1
                charOrDisch=-1                
        elif bat_action < 0:
            charOrDisch=0
            
    elif charOrDisch == -1:
        if bat_action < 0:
            if x[0] == lowBound:
                contDisch+=1
                charOrDisch=1                
        elif bat_action > 0:
            charOrDisch=0
            
    elif charOrDisch == 0:
        if x[0] == upBound:
            charOrDisch=-1
        elif x[0] == lowBound:
            charOrDisch=1
            
    grid_sum=abs((demand[0]-pv_gen[0]+bat_action))
    for i in range(1,dim):
        bat_action = x[i]-x[i-1]
        if bat_action>5:
            bat_action=5
            x[i]=x[i-1]+5
        if bat_action<-5:
            bat_action=-5
            x[i]=x[i-1]-5
            
        if charOrDisch == 1:
            if bat_action > 0:
                if x[i] == upBound:
                    contChar+=1
                    charOrDisch=-1                
            elif bat_action < 0:
                charOrDisch=0
            
        elif charOrDisch == -1:
            if bat_action < 0:
                if x[i] == lowBound:
                    contDisch+=1
                    charOrDisch=1                
            elif bat_action > 0:
                charOrDisch=0
            
        elif charOrDisch == 0:
            if x[i] == upBound:
                charOrDisch=-1
            elif x[i] == lowBound:
                charOrDisch=1
        
        grid_sum+=abs((demand[i]-pv_gen[i]+bat_action))
    
    fitness = const1*(grid_sum)-const2*(contChar+contDisch)
    return fitness, charOrDisch, contChar, contDisch, grid_sum

def costFunctionBatteryLifetime3(x,demand,pv_gen,bat_t0,elprices,battery):
    charOrDisch = battery.contAct
    contChar=0
    contDisch=0
    
    charOrDisch2 = battery.action
    contChar=0
    contDisch=0
    
    numCharChanges=0
    numBatNullState=0
    dim = len(x)
    bat_action = x[0]-bat_t0
    
    if bat_action == 0:
        numBatNullState+=1
        
    if bat_action>5:
        bat_action=5
        x[0]=bat_t0+5
    if bat_action<-5:
        bat_action=-5
        x[0]=bat_t0-5
        
    if charOrDisch2 == 1:
        if bat_action > 0:
            if x[0] == battery.capacity:
                charOrDisch2=-1   
        elif bat_action < 0:
            charOrDisch2=-1
            numCharChanges+=1 
            
    elif charOrDisch2 == -1:
        if bat_action < 0:
            if x[0] == 0.0:
                charOrDisch2=1               
        elif bat_action > 0:
            charOrDisch2=1
            numCharChanges+=1
        
    #continuous
    if charOrDisch == 1:
        if bat_action > 0:
            if x[0] == battery.capacity:
                contChar+=1
                charOrDisch=-1      
        elif bat_action < 0:
            charOrDisch=0
            
    elif charOrDisch == -1:
        if bat_action < 0:
            if x[0] == 0.0:
                contDisch+=1
                charOrDisch=1
        elif bat_action > 0:
            charOrDisch=0
            
    elif charOrDisch == 0:
        if x[0] == battery.capacity:
            charOrDisch=-1
        elif x[0] == 0.0:
            charOrDisch=1
            
    grid_sum=abs((demand[0]-pv_gen[0]+bat_action))
    for i in range(1,dim):
        bat_action = x[i]-x[i-1]
        
        if bat_action == 0:
            numBatNullState+=1
            
        if bat_action>5:
            bat_action=5
            x[i]=x[i-1]+5
        if bat_action<-5:
            bat_action=-5
            x[i]=x[i-1]-5
            
        if charOrDisch2 == 1:
            if bat_action > 0:
                if x[i] == battery.capacity:
                    charOrDisch2=-1   
            elif bat_action < 0:
                charOrDisch2=-1
                numCharChanges+=1 
            
        elif charOrDisch2 == -1:
            if bat_action < 0:
                if x[i] == 0.0:
                    charOrDisch2=1    
            elif bat_action > 0:
                charOrDisch2=1
                numCharChanges+=1
        
        #continuous
        if charOrDisch == 1:
            if bat_action > 0:
                if x[i] == battery.capacity:
                    contChar+=1
                    charOrDisch=-1
            elif bat_action < 0:
                charOrDisch=0
                numCharChanges+=1
        elif charOrDisch == -1:
            if bat_action < 0:
                if x[i] == 0.0:
                    contDisch+=1
                    charOrDisch=1       
            elif bat_action > 0:
                charOrDisch=0
                numCharChanges+=1
            
        elif charOrDisch == 0:
            if x[i] == battery.capacity:
                charOrDisch=-1
            elif x[i] == 0.0:
                charOrDisch=1
        
        grid_sum+=abs((demand[i]-pv_gen[i]+bat_action))
    
    fitness = (grid_sum)+10*(numCharChanges+numBatNullState)
    return fitness, charOrDisch, charOrDisch2, contChar, contDisch, grid_sum, numCharChanges, numBatNullState


In [7]:
class Battery:
    def __init__(self,cap,soc,act,contAct,numContChar,numContDisch):
        self.capacity=cap
        self.SOC=soc
        self.action=act
        self.contAct=contAct
        self.numContChar=numContChar
        self.numContDisch=numContDisch

In [13]:
class HeuristicOptimizer():
    def __init__(self, start, end, batCapacity, batSOC, batContAct, houseNumber):
        global str
        self.houseNumber=houseNumber
        self.totalImport = 0
        self.totalExport = 0
        self.start = start
        self.end = end
        self.battery=Battery(batCapacity, batSOC, 1, batContAct, 0, 0)
        self.data = pd.read_csv('../data/load/dataid' + str(houseNumber) + '.csv', index_col=0, sep=";")
        self.data.loc[self.data['loadProd'] < 0, 'loadProd'] = 0
        
        self.elprices = pd.read_csv('../data/elprices.csv', index_col=0, sep=",")
        self.elprices = pd.DataFrame(self.elprices[['prices']])
        self.elprices = self.elprices['prices']
        self.iter=0
        self.gen = pd.DataFrame(self.data[['loadProd']])
        self.con = pd.DataFrame(self.data[['loadCons']])
        self.grid = pd.DataFrame(self.data[['loadGrid']])
        
        self.newGridProgress=[]
        self.batSocProgress=[]
    
        self.con = self.con['loadCons']
        self.grid = self.grid['loadGrid']
        self.gen = self.gen['loadProd']
        
    def calculateImportExport(self,gridAction):
        if gridAction >= 0:
            self.totalImport+=gridAction
        if gridAction < 0:
            self.totalExport+=(-gridAction)
            
    def calculatePrice(self,elprices,gridActions):
        price=0
        for i in range(0,len(elprices)):
            if gridActions[i] > 0:
                price+=(elprices[i]*gridActions[i])
        return price
    
    def calculateChargingChanges(self,batActionsProgress):
        charOrDisch = 1
        numCharChanges = 0
        numCharges = 1
        numDischarges = 0
        numNullStates=0
        for batAction in batActionsProgress:
            if batAction == 0:
                numNullStates+=1
            if charOrDisch==1 and batAction < 0:
                numCharChanges+=1
                numDischarges+=1
                charOrDisch=-1
            elif charOrDisch==-1 and batAction > 0:
                numCharChanges+=1
                numCharges+=1
                charOrDisch=1
        print("numCharChanges ",numCharChanges)
        print("numCharges ",numCharges)
        print("numDischarges ",numDischarges)
        print("numNullStates ",numNullStates)
        return numCharChanges,numCharges,numDischarges,numNullStates
    
    def calculateChargingChanges2(self,batActionsProgress,batSOCProgress,charOrDisch):
        print('dlzka poli ',len(batActionsProgress),len(batSOCProgress))
        numCharChanges = 0
        numCharges = 1
        numDischarges = 0
        numNullStates=0
        batLifeLoss=0
        dischAction=0
        dischFinish=0
        DODProgress=[]
        DODProgress2=[]
        DOD=0
        avgDOD=0
        avgDOD2=0
        avgSOC=0
        avgDischAction=0
        length = len(batActionsProgress)
        for i in range(0,length):
            batAction=batActionsProgress[i]
            avgSOC+=(batSOCProgress[i]/self.battery.capacity)
            avgDOD2+=(self.battery.capacity-batSOCProgress[i])
            DODProgress2.append((self.battery.capacity-batSOCProgress[i]))
            
            if charOrDisch==1 and batAction < 0:
                numCharChanges+=1
                numDischarges+=1
                charOrDisch=-1
                DOD=(self.battery.capacity-batSOCProgress[i])/self.battery.capacity #vypocet depth of discharge
                dischAction=batAction
                dischFinish=0
                
            elif charOrDisch == -1 and batAction < 0:
                DOD=(self.battery.capacity-batSOCProgress[i])/self.battery.capacity #vypocet depth of discharge
                dischAction+=(batAction)
                dischFinish=0
                
            elif charOrDisch==-1 and batAction > 0:
                numCharChanges+=1
                numCharges+=1
                charOrDisch=1
                batLifeLoss+=(abs(dischAction)*DOD)
                avgDischAction+=abs(dischAction)
                avgDOD+=DOD
                DODProgress.append(DOD)
                dischFinish=1
                
            elif batAction == 0:
                numNullStates+=1
                
        if dischFinish==0:
            batLifeLoss+=(abs(dischAction)*DOD)
            avgDOD+=DOD
            DODProgress.append(DOD)
        if numDischarges>0:
            batLifeLoss=batLifeLoss/numDischarges
            avgDOD=avgDOD/numDischarges
            avgDischAction=avgDischAction/numDischarges
        else:
            batLifeLoss=0
        print("numCharChanges ",numCharChanges)
        print("numCharges ",numCharges)
        print("numDischarges ",numDischarges)
        print("numNullStates ",numNullStates)
        self.battery.numBatNullState=numNullStates
        print("batLifeLoss ",batLifeLoss,batLifeLoss*numDischarges)
        print("avgDOD ",(sum(DODProgress)/len(DODProgress)),len(DODProgress))
        print("avgDOD2 ",(sum(DODProgress2)/len(DODProgress2))/self.battery.capacity)
        print("avgSOC ",(sum(batSOCProgress)/len(batSOCProgress))/self.battery.capacity)
        print("avgDischAction ",avgDischAction,avgDischAction*numDischarges)
        
        totalBat=0
        for i in batActionsProgress:
            totalBat+=abs(i)
        print('totalBat: ',totalBat)
        
        self.battery.batLifeLoss=batLifeLoss*numDischarges
        self.battery.avgDOD=(sum(DODProgress2)/len(DODProgress2))/self.battery.capacity
        self.battery.avgDODPerDisch=(sum(DODProgress)/len(DODProgress))
        self.battery.avgSOC=(sum(batSOCProgress)/len(batSOCProgress))/self.battery.capacity
        self.battery.avgDischAction=avgDischAction
        self.battery.totalBat=totalBat
        
        self.battery.sumDOD=sum(DODProgress2)
        self.battery.sumDODPerDisch=sum(DODProgress)
        self.battery.sumSOC=sum(batSOCProgress)
        self.battery.sumDischAction=avgDischAction*numDischarges
        
                    
    def saveResults(self,gridTotalProgress,batSOCProgress,batActionsProgress,pvToDemandProgress,total_demand, pvToDemandTotal,contAct,numContChar,numContDisch,totalPrice,finalnumCharges,finalnumDischarges,finalnumCharChanges,finalnumNullStates):    
        df=self.data[self.start-1:self.end]
        batSoc = batSOCProgress
        if self.iter==0:
            batSoc.pop(0)
            self.iter=1
        dat1 = pd.DataFrame({'newLoadGrid': gridTotalProgress})
        dat2 = pd.DataFrame({'batterySOC': batSoc})
        dat3 = pd.DataFrame({'batteryActions': batActionsProgress})
        dat4 = pd.DataFrame({'pvUsage': pvToDemandProgress})
        dat5 = pd.DataFrame({'prices': self.elprices[self.start-1:self.end]})
        dat5.index = np.arange(0, len(dat5) + 0)
        dat6 = pd.DataFrame({'batteryLastSOC': [self.battery.SOC]})
        dat7 = pd.DataFrame({'totalPrice': [totalPrice]})
        dat8 = pd.DataFrame({'totalImport': [self.totalImport]})
        dat9 = pd.DataFrame({'totalExport': [self.totalExport]})
        dat10 = pd.DataFrame({'totalCons': [total_demand]})
        dat11 = pd.DataFrame({'totalPvUsage': [pvToDemandTotal]})
        dat12 = pd.DataFrame({'contAct': [contAct]})
        dat26 = pd.DataFrame({'batteryAct': [self.battery.action]})
        dat13 = pd.DataFrame({'numContChar': [numContChar]})
        dat14 = pd.DataFrame({'numContDisch': [numContDisch]})
        
        dat15 = pd.DataFrame({'charges': [finalnumCharges]})
        dat16 = pd.DataFrame({'discharges': [finalnumDischarges]})
        dat17 = pd.DataFrame({'CharChanges': [finalnumCharChanges]})
        dat18 = pd.DataFrame({'batNullStates': [finalnumNullStates]})
        
        dat19 = pd.DataFrame({'batLifeLoss': [self.battery.batLifeLoss]})
        if finalnumDischarges==0:
            dat20 = pd.DataFrame({'batLifeLossPerDisch': [0]})
        else:
            dat20 = pd.DataFrame({'batLifeLossPerDisch': [self.battery.batLifeLoss/finalnumDischarges]})
        dat21 = pd.DataFrame({'avgDOD': [self.battery.avgDOD]})
        dat22 = pd.DataFrame({'avgDODPerDisch': [self.battery.avgDODPerDisch]})
        dat23 = pd.DataFrame({'avgDischAction': [self.battery.avgDischAction]})
        dat24 = pd.DataFrame({'DischAction': [self.battery.avgDischAction*finalnumDischarges]})
        dat25 = pd.DataFrame({'avgSOC': [self.battery.avgSOC]})
        dat27 = pd.DataFrame({'totalBat': [self.battery.totalBat]})
        
        dat28 = pd.DataFrame({'sumDOD': [self.battery.sumDOD]})
        dat29 = pd.DataFrame({'sumDODPerDisch': [self.battery.sumDODPerDisch]})
        dat30 = pd.DataFrame({'sumSOC': [self.battery.sumSOC]})
        dat31 = pd.DataFrame({'sumDischAction': [self.battery.sumDischAction]})
        
        dat1 = pd.concat([dat1, dat2, dat3, dat4, dat5, dat6, dat7, dat8, dat9, dat10,dat11, dat12, dat26, dat13, dat14, dat15, dat16, dat17, dat18, dat19, dat20, dat21, dat22, dat23, dat24, dat25, dat27, dat28, dat29, dat30, dat31], axis=1)
        dat1.index = np.arange(self.start, len(dat1) + self.start)
        df = pd.concat([df, dat1], axis=1)
        df.to_csv("Vysledky/HEMaxBatLife/HEMaxBatLifeData"+str(self.houseNumber)+".csv",sep=";")

    
    def optimizeHeuristic(self):
        print("Heuristic Optimization")
        total_demand = 0
        total_grid = 0
        totalPrice = 0
        
        pvToDemandTotal = 0

        gridTotalProgress = []
        pvToDemandProgress = []
        batSOCProgress = []
        batActionsProgress = []
        batSOCProgress.append(0)
        import time
        zac = int(round(time.time() * 1000))
        print(zac)
        print("start")
        
        conTotal = 0
        gridImport = 0
        gridExport = 0
        pvToMicrogrid = 0
        batToMicrogrid = 0
        batChar = 0
        batDisch = 0
        totalPrice=float(0)
        
        lowBound=self.battery.capacity*0.2
        upBound=self.battery.capacity*0.8
        
        print(lowBound,upBound)
        
        for c in range(self.start, self.end+1):
            pv = 0
            grid = 0
            bat = 0
            conTotal+=self.con[c]
            if self.gen[c] >= self.con[c]:
                pv = self.con[c]
                pvRest = self.gen[c]-self.con[c]
                if self.battery.contAct == 1 and self.battery.SOC < upBound:
                    batRest = upBound-self.battery.SOC
                    if batRest>5:
                        batRest=5
                    if pvRest >= batRest:
                        self.battery.SOC+=batRest
                        grid = -(pvRest-batRest)
                        gridExport+=(pvRest-batRest)
                    else:# pvRest < batRest:
                        self.battery.SOC+=pvRest
                    if self.battery.SOC == upBound:
                        self.battery.numContChar+=1
                        self.battery.contAct=-1  
                else:
                    grid=-pvRest
                    gridExport+=pvRest
            else:# self.gen[c] < self.con[c]:
                pv = self.gen[c]
                grid = self.con[c]-self.gen[c]
                conRest = self.con[c]-self.gen[c]
                if self.battery.contAct==-1:
                    batteryMaxSupply = self.battery.SOC-lowBound
                    if batteryMaxSupply > 5:
                        batteryMaxSupply = 5
                    if batteryMaxSupply >= conRest:
                        self.battery.SOC-=conRest
                        grid=0#=0
                    else:
                        self.battery.SOC-=batteryMaxSupply
                        grid-=batteryMaxSupply
                        conRest-=batteryMaxSupply
                        gridImport+=grid
                    if self.battery.SOC == lowBound:
                        self.battery.numContDisch+=1
                        self.battery.contAct=1   
                else:
                    gridImport+=grid
            bestPos = [grid,pv,bat]
            gridTotalProgress.append(grid)
            batSOCProgress.append(self.battery.SOC)
            
        print('FINAL:')
        print("total consumption ",conTotal)
        print("grid import ",gridImport)
        print("grid export ",gridExport)
        print("grid total ",gridImport-gridExport)
        print("cont act ", self.battery.contAct)
        print("num cont charges ", self.battery.numContChar)
        print("num cont discharges ", self.battery.numContChar)
        print()
        for i in batSOCProgress:
            self.batSocProgress.append(i)
        for i in gridTotalProgress:
            self.newGridProgress.append(i)
        l=len(batSOCProgress)
        for i in range(1,l):
            batActionsProgress.append((batSOCProgress[i]-batSOCProgress[i-1]))
            
        
        algPvToDemand=0
        l=len(gridTotalProgress)
        for i in range(0,l):
            pvRest=0
            algPvToDemandInHour = 0
            if self.gen[i+1] >= self.con[i+1]:
                algPvToDemand += self.con[i+1]
                algPvToDemandInHour += self.con[i+1]
                pvRest = self.gen[i+1]-self.con[i+1]
            else:
                algPvToDemand += self.gen[i+1]
                algPvToDemandInHour += self.gen[i+1]
                    
            if pvRest > 0 and batActionsProgress[i]>0:
                if batActionsProgress[i]>=pvRest:
                    algPvToDemand += pvRest
                    algPvToDemandInHour += pvRest
                else:
                    algPvToDemand += batActionsProgress[i]
                    algPvToDemandInHour += batActionsProgress[i]
            pvToDemandProgress.append(algPvToDemandInHour)
        pvToDemandTotal+=algPvToDemand
            
        print("TotalProgress length ",len(gridTotalProgress),len(batSOCProgress), len(batActionsProgress), len(pvToDemandProgress))
        print('total generation ',sum(self.gen[self.start-1:self.end]),pvToDemandTotal)
        print('pv to demand', sum(pvToDemandProgress))
        
        batSOCProgress.pop(0)
        conWindow=[]
        genWindow=[]
        elprices=[]
        for i in range(self.start,self.end+1):
            conWindow.append(self.con[i])
            genWindow.append(self.gen[i])
            elprices.append(self.elprices[i])
            
            

        self.battery.contAct=0
        var, self.battery.contAct, self.battery.numContChar,self.battery.numContDisch, var3 = contActions2(batSOCProgress,conWindow,genWindow,0,elprices,self.battery,1,1)
        print('cont actions ',self.battery.contAct, self.battery.numContChar,self.battery.numContDisch)
        self.battery.action=1 
        Fnew, charOrDischNew, charOrDisch2New, contCharNew, contDischNew, gridPriceNew, numCharChangesNew, numBatNullStateNew = costFunctionBatteryLifetime3(batSOCProgress,conWindow,genWindow,self.battery.SOC,elprices,self.battery)                #som zmenil
        print('charging changes ',numCharChangesNew,numBatNullStateNew)
        numCharChanges,numCharges,numDischarges,numNullStates = self.calculateChargingChanges(batActionsProgress)
        
        print()
        print("numCharChanges ",numCharChanges)
        print("numCharges ",numCharges)
        print("numDischarges ",numDischarges)
        print("numNullStates ",numNullStates)
        
        self.calculateChargingChanges2(batActionsProgress,batSOCProgress, 1)
        
        totalPrice=self.calculatePrice(elprices,gridTotalProgress)
        self.totalImport=gridImport
        self.totalExport=gridExport
        print('total price ', totalPrice)
        print(self.totalImport,self.totalExport)
        print()
        totalBat=0
        for i in batActionsProgress:
            totalBat+=abs(i)
        
        print('totalBat: ',totalBat)
        self.saveResults(gridTotalProgress, batSOCProgress,batActionsProgress, pvToDemandProgress,conTotal,pvToDemandTotal,self.battery.contAct,self.battery.numContChar,self.battery.numContDisch,totalPrice,numCharges,numDischarges,numCharChanges,numNullStates)
        
        
        

In [12]:
#start=2161,end=5521
ho = HeuristicOptimizer(start=1,end=168,batCapacity=13,batSOC=0,batContAct=1,houseNumber=744)
ho.optimizeHeuristic()


Heuristic Optimization
1590878585055
start
2.6 10.4
FINAL:
total consumption  117.22166666666674
grid import  41.24408333333333
grid export  52.25468333333335
grid total  -11.010600000000018
cont act  -1
num cont charges  5
num cont discharges  5

TotalProgress length  168 169 168 168
total generation  132.66125000000002 80.40656666666668
pv to demand 80.40656666666666
cont actions  -1 4 4
charging changes  12 81
numCharChanges  9
numCharges  5
numDischarges  5
numNullStates  82

numCharChanges  9
numCharges  5
numDischarges  5
numNullStates  82
dlzka poli  168 168
numCharChanges  9
numCharges  5
numDischarges  5
numNullStates  82
batLifeLoss  5.779348974876069 28.896744874380346
avgDOD  0.771861794871795 5
avgDOD2  0.47822382478632497
avgSOC  0.521776175213675
avgDischAction  6.24 31.200000000000003
totalBat:  78.7710166666667
total price  1.039235622833333
41.24408333333333 52.25468333333335

totalBat:  78.7710166666667
